# Part 1: Cross validation

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.api as sm
from ISLP import load_data
from ISLP.models import (ModelSpec as MS,
                         summarize,
                         poly)
from sklearn.model_selection import train_test_split
from functools import partial
from sklearn.model_selection import \
     (cross_validate,
      KFold,
      ShuffleSplit)
from sklearn.base import clone
from ISLP.models import sklearn_sm

In [2]:
import warnings
warnings.filterwarnings('ignore')

In this part we follow the first part of the lab of Section 4 of our textbook to learn how to implement cross validation in Python (see [here](https://islp.readthedocs.io/en/latest/labs/Ch05-resample-lab.html) for the original version of the Lab)


## 1a) Validation set approach

**Objective of part 1a)**: Use the validation set approach to evaluate the performance of a model predicting `mpg` in the `Auto` dataset based on predictor `horsepower`.

In [3]:
# Run this cell to load the dataset
Auto =  load_data('Auto')

### Task 1.1: 
Split the dataset into a train and a test part. Your test set should have 196 samples.

In [4]:
train, validation = train_test_split(Auto, test_size=196)

In [5]:
train

,mpg,cylinders,displacement,horsepower,weight,acceleration,year,origin,name
345,34.1,4,91.0,68,1985,16.0,81,3,mazda glc 4
78,26.0,4,96.0,69,2189,18.0,72,2,renault 12 (sw)
215,30.0,4,111.0,80,2155,14.8,77,1,buick opel isuzu deluxe
375,36.0,4,107.0,75,2205,14.5,82,3,honda accord
329,29.8,4,89.0,62,1845,15.3,80,2,vokswagen rabbit
...,...,...,...,...,...,...,...,...,...
103,12.0,8,400.0,167,4906,12.5,73,1,ford country
280,22.3,4,140.0,88,2890,17.3,79,1,ford fairmont 4
350,33.7,4,107.0,75,2210,14.4,81,3,honda prelude
165,29.0,4,97.0,75,2171,16.0,75,3,toyota corolla


In [6]:
validation

,mpg,cylinders,displacement,horsepower,weight,acceleration,year,origin,name
342,32.3,4,97.0,67,2065,17.8,81,3,subaru
33,16.0,6,225.0,105,3439,15.5,71,1,plymouth satellite custom
71,15.0,8,304.0,150,3892,12.5,72,1,amc matador (sw)
313,24.3,4,151.0,90,3003,20.1,80,1,amc concord
69,13.0,8,400.0,190,4422,12.5,72,1,chrysler newport royal
...,...,...,...,...,...,...,...,...,...
9,15.0,8,390.0,190,3850,8.5,70,1,amc ambassador dpl
7,14.0,8,440.0,215,4312,8.5,70,1,plymouth fury iii
273,17.0,6,163.0,125,3140,13.6,78,2,volvo 264gl
61,13.0,8,350.0,165,4274,12.0,72,1,chevrolet impala


### Task 1.2:
Train a simple linear regression model on the training set which predicts `mpg` based on the unique predictor `horsepower`. Make sure to use the training set to train your model.

In [7]:
design = MS(['horsepower']).fit(train)
X_train = design.transform(train)
X_train

,intercept,horsepower
345,1.0,68
78,1.0,69
215,1.0,80
375,1.0,75
329,1.0,62
...,...,...
103,1.0,167
280,1.0,88
350,1.0,75
165,1.0,75


In [8]:
y_train = train.mpg

In [9]:
model = sm.OLS(y_train,X_train)
results = model.fit()
summarize(results)

,coef,std err,t,P>|t|
intercept,38.2704,1.025,37.319,0.0
horsepower,-0.1460,0.009,-15.864,0.0


### Task 1.3: 
Now use the validation set to compute an estimate for the model's MSE (mean squared error).

In [10]:
y_valid_actual = validation.mpg
X_validation = design.transform(validation)
y_valid_predicted = results.predict(X_validation)
MSE = np.mean((y_valid_actual - y_valid_predicted)**2)
MSE

25.40572509811524

### Task 1.4:
Below you find a preimplemented function `evalMSE()`. Explain what this function does (what are the arguments, what is the output)?

In [11]:
def evalMSE(predictors,
           train,
           validation):
    # build design matrix and response vector
    design = MS(predictors).fit(train)
    X_train = design.transform(train)
    y_train = train.mpg 

    # train model
    model = sm.OLS(y_train,X_train)
    results = model.fit()

    # compute MSE on the validation set
    y_valid_actual = validation.mpg
    X_validation = design.transform(validation)
    y_valid_predicted = results.predict(X_validation)
    MSE = np.mean((y_valid_actual - y_valid_predicted)**2)
    return MSE

In [12]:
evalMSE(['horsepower'], train, validation)

25.40572509811524

*Solution*: The function takes a set of predictors, a dataframe of training data and a dataframe of validation data as inputs. Based on these it trains a linear regression model on the training data using the predictors provided as arguments. It then computes the test MSE on the provided validation set. This test MSE is then returned as output.

### Task 1.5:

Use the function `evalMSE()` to estimate the MSE on the validation set for linear regression models including successively higher polynomial terms of `horsepower` ranging from degree 1 to degree 3.

In [13]:
from ISLP.models import poly
MSE = []
for i in range(1,4):
    predictors = [poly('horsepower', i)] # choose powers of horsepower as predictors up to degree i+1
    err = evalMSE(predictors, train, validation)
    MSE.append(err)

In [14]:
MSE

[25.40572509811526, 19.186027859894136, 19.11569158811723]

## 1b) Cross validation

Cross validation is implemented most comfortably in `scikit-learn`. In order to use the `scikit-learn` implementation of cross validation with our `statsmodels` linear model, we use the wrapper `sklearn_sm` provided by the `ISLP` library. From the lab in Chapter 5:

"The class `sklearn_sm()` has as its first argument a model from `statsmodels`. It can take two additional optional arguments: `model_str` which can be used to specify a formula, and `model_args` which should be a dictionary of additional arguments used when fitting the model. For example, to fit a logistic regression model we have to specify a family argument. This is passed as `model_args={'family':sm.families.Binomial()}`."

After specifying our design matrix `X` and the vector `y` we call the `scikit-learn` function `cross_validate` (see [documentation](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.cross_validate.html)).

The result is a dictionary which among others contains the `test_score` which we are interested when we use cross validation to estimate the test error.

### Task 1.6:
Run the following cell to initialize an `sklearn_sm` object with a `statsmodels.OLS` model and a `ModelSpec` object with specified predictor `horsepower`, with dataset `X` with removed `mpg` column und with response vector `y`.

In [15]:
model = sklearn_sm(sm.OLS,
                  MS(['horsepower']))
X = Auto.drop(columns = ['mpg'])
y = Auto['mpg']

The function `cross_validate` is the original `scikit-learn` function which carries out cross validation. We provide the following arguments:
- a model which needs `fit()` and `predict()` methods
- a design matrix `X` and a vector of training labels `y`
- the parameter `cv` specifying the number of folds for cross validation.

See the [official documentation](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.cross_validate.html) for more details.

### Task 1.7:
Run a Leave-one-out-cross-validation for a simple linear regression model predicting `mpg` based on `horsepower`. To do so, choose as many folds as your dataset has samples using the parameter `cv`.

In [16]:
cv_results = cross_validate(model,
                           X,
                           y,
                           cv = Auto.shape[0])

In [17]:
cv_err = np.mean(cv_results['test_score'])
cv_err

24.23151351792922

### Task 1.8:

We can repeat this procedure to compare different models. In the following we do this with various polynomial fits.

Compute the LOOCV error for models to predict `mpg` based on polynomial powers of `mpg` for degrees 1 to 5.

In [18]:
cv_error = np.zeros(5) #initialize empty vector for CV errors of 5 models
M = sklearn_sm(sm.OLS)
y = Auto['mpg']

# loop over 5 different polynomial models
for i in range(1,6):
    # choose power of horsepowers as predictors up to degree i+1
    predictors = [poly('horsepower', i)]
    X = MS(predictors).fit_transform(Auto)
    #perform cross validation for polynomial model of degree i+i
    cv_results = cross_validate(M,
                               X,
                               y,
                               cv = Auto.shape[0])
    # save cross validation error into result vector
    cv_error[i-1] = np.mean(cv_results['test_score'])
cv_error

array([24.23151352, 19.24821312, 19.33498406, 19.42443031, 19.03321385])

### Task 1.9:

Instead of using $K = n$ folds such as above (resulting in Leave-One-Out-Cross-Validation (LOOCV)) we can also specify a smaller integer $K$ of folds. There are two possibilities for this:
- set `cv = K`,
- specify a cross validation generator such as `KFold` (see [documentation](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.KFold.html))

It is recommended to use the second approach, which is generally preferred as we do better control the kind of split when using `KFold`.

Compute the cross validation error for models to predict `mpg` based on polynomial powers of `mpg` for degrees 1 to 5. Use 10 folds.

In [19]:
cv_error = np.zeros(5)

cross_val = KFold(n_splits = 10,
                 shuffle = True,
                 random_state = 42)

M = sklearn_sm(sm.OLS)
y = Auto['mpg']

# loop over 5 different polynomial models
for i in range(5):
    # choose power of horsepowers as predictors up to degree i+1
    predictors = [poly('horsepower', i+1)]
    X = MS(predictors).fit_transform(Auto)
    #perform cross validation for polynomial model of degree i+i
    cv_results = cross_validate(M,
                               X,
                               y,
                               cv = cross_val)
    # save cross validation error into result vector
    cv_error[i] = np.mean(cv_results['test_score'])
cv_error

array([24.1998082 , 19.22863661, 19.26626535, 19.35109227, 19.02323325])

# Part 2: Case study cross validation (no solutions beyond this point as we did not get further in lecture)

(see Exercise 5.4.5)

In this case study we use the credit card dataset to predict the probability of default. We will build a logistic regression model and estimate its test error using the validation set approach and the cross-validation approach.

In [20]:
# run this cell to load the data
Default = load_data('Default')
Default

,default,student,balance,income
0,No,No,729.526495,44361.625074
1,No,Yes,817.180407,12106.134700
2,No,No,1073.549164,31767.138947
3,No,No,529.250605,35704.493935
4,No,No,785.655883,38463.495879
...,...,...,...,...
9995,No,No,711.555020,52992.378914
9996,No,No,757.962918,19660.721768
9997,No,No,845.411989,58636.156984
9998,No,No,1569.009053,36669.112365


Background information on the dataset can be found [in the documentation](https://islp.readthedocs.io/en/latest/datasets/Default.html).

## Task 2.1
Fit a logistic regression model that uses `income` and `balance` to predict `default`.

In [21]:
predictors = Default.columns.drop(['default','student'])
design = MS(predictors).fit(Default)
X = design.transform(Default)
y = Default.default.map(
    {'No': 0,
    'Yes': 1}
)

model = sm.Logit(y,X)
results = model.fit()
results.summary()

Optimization terminated successfully.
         Current function value: 0.078948
         Iterations 10


<class 'statsmodels.iolib.summary.Summary'>
"""
                           Logit Regression Results                           
==============================================================================
Dep. Variable:                default   No. Observations:                10000
Model:                          Logit   Df Residuals:                     9997
Method:                           MLE   Df Model:                            2
Date:                Mon, 08 Jul 2024   Pseudo R-squ.:                  0.4594
Time:                        23:45:50   Log-Likelihood:                -789.48
converged:                       True   LL-Null:                       -1460.3
Covariance Type:            nonrobust   LLR p-value:                4.541e-292
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
intercept    -11.5405      0.435    -26.544      0.000     -12.393     -10.688
balance        0.0056      0.000     24.835      0.000       0.005       0.006
income      2.081e-05   4.99e-06      4.174      0.000     1.1e-05    3.06e-05
==============================================================================

Possibly complete quasi-separation: A fraction 0.14 of observations can be
perfectly predicted. This might indicate that there is complete
quasi-separation. In this case some parameters will not be identified.
"""

## Task 2.2
Using the validation set approach, estimate the test error of this model. In order to do this, you must perform the following steps:

i. Split the sample set into a training set and a validation set.

ii. Fit a multiple logistic regression model using only the training observations.

iii. Obtain a prediction of default status for each individual in the validation set by computing the posterior probability of default for that individual, and classifying the individual to the default category if the posterior probability is greater than 0.5.

iv. Compute the validation set error, which is the fraction of the observations in the validation set that are misclassified.

In [22]:
# i. splitting into validation and test set
train, val = train_test_split(
    Default,
    test_size=0.3,
    random_state = 42
)

In [23]:
# ii. fit logistic regression model using only training observations
predictors = train.columns.drop(['default','student'])
design2 = MS(predictors).fit(train)
X2 = design2.transform(train)
y2 = train.default.map(
    {'No': 0,
    'Yes': 1}
)

model2 = sm.Logit(y2,X2)
results2 = model2.fit()
results2.summary()

Optimization terminated successfully.
         Current function value: 0.078256
         Iterations 10


<class 'statsmodels.iolib.summary.Summary'>
"""
                           Logit Regression Results                           
==============================================================================
Dep. Variable:                default   No. Observations:                 7000
Model:                          Logit   Df Residuals:                     6997
Method:                           MLE   Df Model:                            2
Date:                Mon, 08 Jul 2024   Pseudo R-squ.:                  0.4743
Time:                        23:45:50   Log-Likelihood:                -547.79
converged:                       True   LL-Null:                       -1042.0
Covariance Type:            nonrobust   LLR p-value:                2.273e-215
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
intercept    -11.6019      0.522    -22.236      0.000     -12.624     -10.579
balance        0.0057      0.000     20.976      0.000       0.005       0.006
income      1.872e-05   5.92e-06      3.163      0.002    7.12e-06    3.03e-05
==============================================================================

Possibly complete quasi-separation: A fraction 0.15 of observations can be
perfectly predicted. This might indicate that there is complete
quasi-separation. In this case some parameters will not be identified.
"""

In [24]:
# iii. Prediction for validation set
X_val = design2.transform(val)
predicted_probs = results2.predict(X_val)
predicted_labels = np.where(predicted_probs > 0.5, 'Yes', 'No')

In [25]:
# iv. validation set error
true_labels = val.default
val_err = 1 - np.mean(predicted_labels == true_labels)
val_err

0.026666666666666616

In [26]:
from ISLP import confusion_table
confusion_table(predicted_labels, true_labels)

Truth,No,Yes
Predicted,,
No,2896,70
Yes,10,24


## Task 2.3
Repeat the process in (b) three times, using three different splits of the observations into a training set and a validation set. Comment on the results obtained.

In [27]:
# your code here
val_errors = np.zeros(3)
for i in range(3):
    train, val = train_test_split(
        Default,
        test_size=0.3
    )
    predictors = train.columns.drop(['default','student'])
    design3 = MS(predictors).fit(train)
    X3 = design3.transform(train)
    y3 = train.default.map(
        {'No': 0,
        'Yes': 1}
    )

    model3 = sm.Logit(y3,X3)
    results3 = model3.fit()
    X_val = design3.transform(val)
    predicted_probs = results2.predict(X_val)
    predicted_labels = np.where(predicted_probs > 0.5, 'Yes', 'No')
    true_labels = val.default
    val_err = 1 - np.mean(predicted_labels == true_labels)
    val_errors[i] = val_err
val_errors

Optimization terminated successfully.
         Current function value: 0.076536
         Iterations 10
Optimization terminated successfully.
         Current function value: 0.080738
         Iterations 10
Optimization terminated successfully.
         Current function value: 0.078245
         Iterations 10


array([0.02633333, 0.02533333, 0.026     ])

**Comment**: The fraction of observations which are misclassified varies slightly due to the fact that we use different validation sets (no random seed set!) in each of the three iterations.

## Task 2.4
Now predict the test error of the model using 10-fold cross-validation. To do so, follow the steps we developed in Part 1b) of this notebook.

**Note**: To carry out this task, we need to define our own custom score using [`sklearn.make_scorer()`](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.make_scorer.html) and pass this to the function [`sklearn.cross_validate()`](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.cross_validate.html) by specifying it as an argument to the `scoring` parameter.

Our custom scoring function needs to have the signature `score_func(y, y_pred, **kwargs)` with `y` being the true labels and `y_pred` the predicted labels as output by `sm.Logit()` when applying the `predict()` method. Since `statsmodels` outputs probabilities rather than actual labels, we first transform these probabilities  into labels. This is what our scoring function `accuracy_score_sm()` does.

In [28]:
# Step 1: Define custom scorer which computes accuracy based on probabilities as
# output by statsmodels
from sklearn.metrics import make_scorer
def accuracy_score_sm(y_true, y_pred_prob):
    # computes the accuracy for the output of a binary classification model

    # inputs:
    #    y_true: ground truth labels, encoded as 0,1
    #    y_pred_prob: probabilities for label 1 as predicted by the model

    # output:
    #    percentage of models 
    y_pred = np.round(y_pred_prob)
    return np.mean(y_true == y_pred)
accuracy_sm = make_scorer(accuracy_score_sm)

# Step 2: Initialize splitter for cross validation and model
cross_val = KFold(n_splits = 10,
                 shuffle = True,
                 random_state = 42)

M = sklearn_sm(sm.Logit)

# Step 3: Define response variable and design matrix for cross validation
y = Default['default'].map({
    'No' : 0,
    'Yes' : 1
})
predictors = ['income', 'balance']
X = MS(predictors).fit_transform(Default)

# Step 4: Run cross validation
cv_results = cross_validate(M,
                            X,
                            y,
                            scoring = accuracy_sm,
                            cv = cross_val)
print('Mean accuracy: ', np.mean(cv_results['test_score']))

Optimization terminated successfully.
         Current function value: 0.077489
         Iterations 10
Optimization terminated successfully.
         Current function value: 0.077985
         Iterations 10
Optimization terminated successfully.
         Current function value: 0.080798
         Iterations 10
Optimization terminated successfully.
         Current function value: 0.076904
         Iterations 10
Optimization terminated successfully.
         Current function value: 0.081253
         Iterations 10
Optimization terminated successfully.
         Current function value: 0.079803
         Iterations 10
Optimization terminated successfully.
         Current function value: 0.078557
         Iterations 10
Optimization terminated successfully.
         Current function value: 0.078019
         Iterations 10
Optimization terminated successfully.
         Current function value: 0.077491
         Iterations 10
Optimization terminated successfully.
         Current function value: 0.

## Task 2.5
Now consider a logistic regression model that predicts the probability of default using income, balance, and a dummy variable for student. Estimate the test error for this model using 10-fold cross-validation. Comment on whether or not including a dummy variable for student leads to a reduction in the test error rate.

In [29]:
# Step 1: Initialize splitter for cross validation and model
# reuse the cross validator from Task 1.4

# Step 2: Define response variable and design matrix for cross validation
# reuse y from Task 1.4
predictors=['income', 'balance', 'student']
X = MS(predictors).fit_transform(
    Default.astype({'student': 'category'}))

# Step 3: Run cross validation
cv_results = cross_validate(M,
                           X,
                           y,
                           scoring = accuracy_sm,
                           cv = cross_val)

print('Mean accuracy: ', np.mean(cv_results['test_score']))

Optimization terminated successfully.
         Current function value: 0.077174
         Iterations 10
Optimization terminated successfully.
         Current function value: 0.077718
         Iterations 10
Optimization terminated successfully.
         Current function value: 0.080384
         Iterations 10
Optimization terminated successfully.
         Current function value: 0.076316
         Iterations 10
Optimization terminated successfully.
         Current function value: 0.080834
         Iterations 10
Optimization terminated successfully.
         Current function value: 0.079514
         Iterations 10
Optimization terminated successfully.
         Current function value: 0.078307
         Iterations 10
Optimization terminated successfully.
         Current function value: 0.077439
         Iterations 10
Optimization terminated successfully.
         Current function value: 0.077127
         Iterations 10
Optimization terminated successfully.
         Current function value: 0.

**Interpretation**: There is no evidence that adding student as additional predictor helps to improve the model accuracy. Hence, we would decide based on this analysis to leave student out.